In [1]:
import torch
import torch.optim as optim
from src.pytorch_efficientnet import EfficientNetwork, augment, augmented_pred
from src.train_test_functions import train_model
import json
import numpy as np
from src.torch_dataset import ZootrDataset 
import os
import pandas as pd

## Set constants

In [2]:
BATCH_SIZE = 20
NUM_WORKERS = 0
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.1
EPOCHS = 10

In [3]:
with open('config/paths_dict.json') as handle:
    paths_dict = json.loads(handle.read())

ITEM_PATH = paths_dict['item_path']
NON_ITEM_PATH = paths_dict['non_item_path']

In [4]:
load_data_objects = False
save_data_objects = True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set up data loaders

In [6]:
if  load_data_objects:
    torch.load('Models/train_data.pth')
    torch.load('Models/valid_data.pth')
else:
    items_images = [ITEM_PATH + i for i in os.listdir(ITEM_PATH)]
    #Get all items
    non_items_images = [NON_ITEM_PATH + i for i in os.listdir(NON_ITEM_PATH)]
    #Get non items
    all_images = list(np.random.permutation(items_images + non_items_images))
    #Combine all images' paths and shuffle
    train_data = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augment)
    train_data_eval = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augmented_pred)
    valid_data = ZootrDataset(all_images[int(np.ceil(len(all_images)*0.8)):], transform_=augmented_pred)
    # Data Objects
    if save_data_objects:
        torch.save(train_data, 'Models/train_data.pth')
        torch.save(valid_data, 'Models/valid_data.pth')

In [7]:
data = {'train': all_images[0:int(np.ceil(len(all_images)*0.7))], 
        'val': all_images[int(np.ceil(len(all_images)*0.7)):int(np.ceil(len(all_images)*0.8))], 
        'test': all_images[int(np.ceil(len(all_images)*0.8)):]}
transformations = {'train': augment, 'val': augmented_pred, 'test': augmented_pred}
dataset_sizes = {'train': int(np.ceil(len(all_images)*0.7)), 'val': int(np.ceil(len(all_images)*0.8)) - int(np.ceil(len(all_images)*0.7)), 
                 'test': len(all_images) - int(np.ceil(len(all_images)*0.8))}

datasets = {x: ZootrDataset(data[x], transform_=transformations[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, drop_last=True, shuffle=False) for x in ['train', 'val', 'test']}

dics = {
    'dataloaders': dataloaders,
    'dataset_sizes': dataset_sizes
}

## Model

In [8]:
model = EfficientNetwork(output_size=2, b1=False, b2=True)
# Criterion
criterion = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                                       patience=1, factor=0.4)

Loaded pretrained weights for efficientnet-b2


In [ ]:
mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=25)

Epoch 1


100%|██████████| 44/44 [00:28<00:00,  1.55it/s]


Epoch 1 loss : 0.49571586373661247


100%|██████████| 6/6 [00:03<00:00,  1.52it/s]


Epoch 1 loss : 0.49778861925005913

Epoch 2


...

100%|██████████| 44/44 [00:26<00:00,  1.65it/s]


Epoch 17 loss : 0.004558537801325916


100%|██████████| 6/6 [00:03<00:00,  1.61it/s]


Epoch 17 loss : 0.07681099741603248

Epoch 18


 93%|█████████▎| 41/44 [00:24<00:01,  1.57it/s]

## Get test metrics

In [ ]:
test_loader = dataloaders['test']

all_preds = torch.empty(0, 2)
all_labels = []
for imgs, label in test_loader:
    imgs_pred = mdl(imgs)

    all_preds = torch.cat((all_preds, imgs_pred))
    all_labels += label
labels = [i.item() for i in all_labels]
#Get all labels

In [ ]:
labels

tensor([[ 3.6693, -3.5890],
        [ 2.7760, -2.5543],
        [ 2.2921, -2.4615],
        [ 3.5099, -3.6109],
        [ 3.4244, -3.6544],
        [ 4.6998, -4.9165],
        [ 3.0694, -3.0519],
        [ 3.2540, -2.5769],
        [ 0.7936, -0.9730],
        [ 5.5266, -5.8216],
        [ 4.5796, -4.5791],
        [ 2.1872, -2.1209],
        [ 3.2225, -3.7740],
        [ 2.2409, -2.5059],
        [ 4.1947, -3.9768],
        [ 4.7298, -4.8547],
        [ 4.4352, -4.9141],
        [ 0.4868, -0.8611],
        [ 3.1692, -3.4783],
        [ 4.8214, -4.7063]], grad_fn=<AddmmBackward0>)

In [ ]:
preds_prob = torch.softmax(all_preds, dim=1)
preds_prob = pd.DataFrame(preds_prob.detach().numpy())
preds_prob.columns = ['False', 'True']
preds_prob['Label'] = labels

In [ ]:
preds_prob.sort_values('True', ascending=False)

,False,True,Label
197,0.000085,0.999915,1
28,0.000123,0.999877,1
202,0.000162,0.999838,1
155,0.000180,0.999820,1
47,0.000190,0.999810,1
...,...,...,...
205,0.999983,0.000017,0
25,0.999984,0.000016,0
229,0.999988,0.000012,0
216,0.999995,0.000005,0


In [ ]:
from sklearn.metrics import roc_auc_score

preds_prob[preds_prob['Label'] == 1].sort_values('True', ascending=False)
roc_auc_score(preds_prob['Label'], preds_prob['True'])

0.9625237886107451

In [ ]:
import numpy as np

print(np.mean(preds_prob.loc[preds_prob['True'] >= 0.9, 'Label']))
print(np.mean(preds_prob.loc[preds_prob['True'] < 0.9, 'Label']))

0.8888888888888888
0.04225352112676056


In [ ]:
#torch.save(mdl, 'model.pth')